In [1]:
# Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings 

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

pd.set_option('display.max_columns', None)

In [3]:
# LOAD DATASET
DATA_PATH = '../data/'

orders = pd.read_csv(DATA_PATH + 'olist_orders_dataset.csv')
order_items = pd.read_csv(DATA_PATH + 'olist_order_items_dataset.csv')
customers = pd.read_csv(DATA_PATH + 'olist_customers_dataset.csv')
payments = pd.read_csv(DATA_PATH + 'olist_order_payments_dataset.csv')
reviews = pd.read_csv(DATA_PATH + 'olist_order_reviews_dataset.csv')
products = pd.read_csv(DATA_PATH + 'olist_products_dataset.csv')

In [ ]:
print("="*80)
print('Data info: orders')
print("="*80)

orders.info()

Data info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [10]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26


In [5]:
# change date time 

date_cols = ['order_purchase_timestamp', 'order_approved_at', 
             'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']

for col in date_cols:
    orders[col] = pd.to_datetime(orders[col])


In [8]:
# Only work with delivered orders

orders_delivered = orders[orders['order_status'] == 'delivered'].copy()

In [9]:
orders_delivered.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26


In [11]:
print(f"Total delivered orders: {len(orders_delivered)}")

print(f"unique customers: {orders_delivered['customer_id'].nunique()}")

Total delivered orders: 96478
unique customers: 96478


In [13]:
# Analysis reference date
ANALYSIS_DATE = orders_delivered['order_purchase_timestamp'].max() + pd.Timedelta(days=1)
CHURN_THRESHOLD_DAYS = 180

print(f"\nAnalysis Date: {ANALYSIS_DATE}")
print(f"Churn Threshold: {CHURN_THRESHOLD_DAYS} days")


Analysis Date: 2018-08-30 15:00:37
Churn Threshold: 180 days


In [14]:
# =============================================================================
# 2. RFM ANALYSIS
# =============================================================================

print("\n" + "="*80)
print("RFM ANALYSIS")
print("="*80)

# payment merge with Order items
order_value = order_items.groupby('order_id').agg({
    'price': 'sum',
    'freight_value': 'sum'
}).reset_index()

order_value['total_value'] = order_value['price'] + order_value['freight_value']


RFM ANALYSIS


In [15]:
# Orders_delivered merge with order_value
orders_with_value = orders_delivered.merge(
    order_value[['order_id', 'total_value']], 
    on='order_id', 
    how='left'
)

In [17]:
# RFM metrikleri hesapla
rfm = orders_with_value.groupby('customer_id').agg({
    'order_purchase_timestamp': lambda x: (ANALYSIS_DATE - x.max()).days,  # Recency
    'order_id': 'count',  # Frequency
    'total_value': 'sum'  # Monetary
}).reset_index()

rfm.columns = ['customer_id', 'recency', 'frequency', 'monetary']

print(rfm.head())

print("\nRFM Statistics:")
print(rfm.describe())

                        customer_id  recency  frequency  monetary
0  00012a2ce6f8dcda20d059ce98491703      288          1    114.74
1  000161a058600d5901f007fab4c27140      410          1     67.41
2  0001fd6190edaaf884bcaf3d49edf079      548          1    195.42
3  0002414f95344307404f0ace7a26f1d5      379          1    179.35
4  000379cdec625522490c315e70c7a9fb      150          1    107.01

RFM Statistics:
            recency  frequency      monetary
count  96478.000000    96478.0  96478.000000
mean     240.121364        1.0    159.826839
std      152.836972        0.0    218.794219
min        1.000000        1.0      9.590000
25%      116.000000        1.0     61.850000
50%      221.000000        1.0    105.280000
75%      350.000000        1.0    176.260000
max      714.000000        1.0  13664.080000


In [18]:
# RFM Score hesapla (1-5 arası)
rfm['r_score'] = pd.qcut(rfm['recency'], q=5, labels=[5,4,3,2,1])
rfm['f_score'] = pd.qcut(rfm['frequency'].rank(method='first'), q=5, labels=[1,2,3,4,5])
rfm['m_score'] = pd.qcut(rfm['monetary'], q=5, labels=[1,2,3,4,5])

# RFM Score birleştir
rfm['rfm_score'] = (rfm['r_score'].astype(str) + 
                    rfm['f_score'].astype(str) + 
                    rfm['m_score'].astype(str))

print(f"\nTop 10 RFM Scores:")
print(rfm['rfm_score'].value_counts().head(10))


Top 10 RFM Scores:
rfm_score
141    875
232    861
212    860
252    858
435    845
524    834
242    833
423    832
424    831
142    823
Name: count, dtype: int64
